# Install MLFlow & the DagsHub python client

In [ ]:
%pip install -q dagshub 'mlflow>=2,<3' ultralytics codecarbon

# Use the DagsHub client to setup connection information for MLflow

In [ ]:
import dagshub
dagshub.init(repo_owner='laia.villagrasa', repo_name='TAED2_SignAI', mlflow=True)

# Use MLflow to log params and metrics

In [ ]:
import os
from datetime import datetime
from pathlib import Path
from ultralytics import YOLO
from codecarbon import EmissionsTracker
import pandas as pd

import mlflow

DATA_YAML = "/mnt/data/dataset.yaml" 
RUN_NAME = f"yolov8-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
MODEL_NAME = "yolov8n.pt" 
IMGSZ = 250                
EPOCHS = 50
BATCH = 16
DEVICE = 0
PROJECT_DIR = "runs"


In [ ]:
model = YOLO(MODEL_NAME)

with mlflow.start_run(run_name=RUN_NAME):
    
    mlflow.log_param("model_base_or_checkpoint", MODEL_NAME)
    mlflow.log_param("imgsz", IMGSZ)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("batch", BATCH)
    mlflow.log_param("data_yaml", DATA_YAML)

    tracker = EmissionsTracker(
        project_name="traffic-signs-yolov8",
        output_dir=str(Path.cwd() / "codecarbon_out"),
        save_to_file=True
    )
    tracker.start()

    results = model.train(
        data=DATA_YAML,
        imgsz=IMGSZ,
        epochs=EPOCHS,
        batch=BATCH,
        device=DEVICE,
        project=PROJECT_DIR,   
        name=RUN_NAME,         
        exist_ok=True
    )

    emissions_kg = tracker.stop()
    mlflow.log_metric("emissions_kg", emissions_kg)

    cc_dir = Path("codecarbon_out")
    if cc_dir.exists():
        for p in cc_dir.glob("*.csv"):
            mlflow.log_artifact(str(p), artifact_path="codecarbon")

    run_dir = Path(PROJECT_DIR) / "detect" / RUN_NAME
    results_csv = run_dir / "results.csv"
    results_yaml = run_dir / "results.yaml"
    best_weights = run_dir / "weights" / "best.pt"

    if results_csv.exists():
        mlflow.log_artifact(str(results_csv), artifact_path="ultralytics")
    if results_yaml.exists():
        mlflow.log_artifact(str(results_yaml), artifact_path="ultralytics")
    if best_weights.exists():
        mlflow.log_artifact(str(best_weights), artifact_path="weights")

    df = pd.read_csv(results_csv)
    last = df.iloc[-1].to_dict()

    for k in ["metrics/precision(B)", "metrics/recall(B)", "metrics/mAP50(B)", "metrics/mAP50-95(B)",                
              "val/box_loss", "val/cls_loss", "val/dfl_loss"]:
        if k in last:
          mlflow.log_metric(k.replace("(B)", ""), float(last[k]))

    mlflow.log_param("optimizer", "SGD (por defecto YOLO)")



Or turn on autologging for most popular ML frameworks,
then run your training code without any modification!

For more info and list of supported frameworks, see:
[https://mlflow.org/docs/latest/tracking.html#automatic-logging](https://mlflow.org/docs/latest/tracking.html#automatic-logging)

In [ ]:
mlflow.autolog()
